### Es sólo la prueba de corrida sin vaciar el análisis al Notebook.

#### Which option should I run?
Options are:

0 = run all data and MC one after another

1 = run data only

2,3 = run MC samples only

In [1]:
int option=1; //option
cout << "Option: " << option;

Option: 1

#### Should I use PROOF? (will make things faster)
Options are:

0 = NO

1 = YES

In [2]:
int parallel=1; //option
cout << "Option: " << parallel;

Option: 1

In [3]:
%%cpp -d
#include "main_HZZAnalysis.cpp"

In [4]:
main_HZZAnalysis(parallel,option);

 +++ Starting PROOF-Lite with 2 workers +++


Opening connections to workers: OK (2 workers)                 


PROOF set to parallel mode (2 workers)


Setting up worker servers: OK (2 workers)                 


Info in <TProofLite::SetQueryRunning>: starting query: 1
Info in <TProofQueryResult::SetRunning>: nwrks: 2
Looking up for exact location of files: OK (4 files)                 
Looking up for exact location of files: OK (4 files)                 
Info in <TPacketizer::TPacketizer>: Initial number of workers: 2
Validating files: OK (4 files)                 


Writting with name option: data 


[TProof::Progress] Total 832 events	|>...................| 0.00 %
Thread 10 (Thread 0x7f9d35793700 (LWP 7050)):
#0  0x00007f9d5c32703f in __GI___select (nfds=0, readfds=0x0, writefds=0x0, exceptfds=0x0, timeout=0x7f9d357927b0) at ../sysdeps/unix/sysv/linux/select.c:41
#1  0x00000000005bbe0a in ?? ()
#2  0x000000000050a94c in ?? ()
#3  0x000000000050c5b9 in _PyEval_EvalFrameDefault ()
#4  0x0000000000509d48 in ?? ()
#5  0x000000000050aa7d in ?? ()
#6  0x000000000050c5b9 in _PyEval_EvalFrameDefault ()
#7  0x0000000000509d48 in ?? ()
#8  0x000000000050aa7d in ?? ()
#9  0x000000000050c5b9 in _PyEval_EvalFrameDefault ()
#10 0x0000000000509455 in _PyFunction_FastCallDict ()
#11 0x0000000000595311 in ?? ()
#12 0x00000000005a067e in PyObject_Call ()
#13 0x00000000005e1b72 in ?? ()
#14 0x0000000000631f44 in ?? ()
#15 0x00007f9d5bff86db in start_thread (arg=0x7f9d35793700) at pthread_create.c:463
#16 0x00007f9d5c33188f in clone () at ../sysdeps/unix/sysv/linux/x86_64/clone.S:95

Thread 9 (Thread